In [1]:

%reload_ext autoreload
%autoreload 2
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["LANGCHAIN_SESSION"] = "session418"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

from langchain import  SerpAPIWrapper
from langchain.agents import Tool, load_tools
from google_maps.google_maps_tool import TravelTimeInfo
from google_maps.google_maps_api import GoogleMapsAPIWrapper

search = SerpAPIWrapper()
search_tool = Tool(
        name="Option search",
        func=search.run,
        description="useful for when you need find businesses or places near a locations. Input: a type of business followed by the location. Output: a list of businesses or places that match the query. Example input: 'coffee shops in San Francisco'",
    )
travel_time_tool = TravelTimeInfo(api_wrapper=GoogleMapsAPIWrapper())
tools = [
    search_tool,
    travel_time_tool,
] + load_tools(['human','llm-math'], llm=llm)

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import BaseLLM
from langchain.output_parsers import CommaSeparatedListOutputParser

class MetaSheetChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        template = (
            "You are an AI assistant helping a travel agent list options for a given time slot in a travel itinerary."
            " Your job is to create a spreadsheet with columns that helps the travel agent compare options given a request."
            " You must have column for all important deatils about an option that would allow for the agent to make an informed recommendation."
            " \nYou have these tools available to add data to the columns: {tools}."
            " \nPlease list the columns you would want to create as a comma sperated list of strings."
            " Be as specific as possible about the columns, using full sentences to describe the data they should contain such as units."
            " Here is the request you are creating columns for: {request}.")


        prompt = PromptTemplate(
            template=template,
            input_variables=[
                "tools",
                "request",
            ],
            output_parser=CommaSeparatedListOutputParser()
        )

        return cls(prompt=prompt, llm=llm, verbose=verbose)
    
class SearchPromptChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        template = """
            Please return a list of search prompts that you would run to get the results for the request: '{request}'
            Each prompt should follow the following format: a type of business (ex: restuarnt, coffee shop, museum, etc) followed by a single location. 
            Example input: 'coffee shops in San Francisco'
            Only return a comman seperated list of prompts following that exact format.
        """


        prompt = PromptTemplate(
            template=template,
            input_variables=[
                "tools",
                "request",
            ],
            output_parser=CommaSeparatedListOutputParser()
        )

        return cls(prompt=prompt, llm=llm, verbose=verbose)

chain = MetaSheetChain.from_llm(llm=llm, verbose=True)

tools_list=", ".join([t.name +": " + t.description +" /n" for t in tools])
request="I need restaurants in for a timeslot between 175 Haight St and 3150 18th St at 12pm that I can walk between"
# result_columns = chain.predict_and_parse(tools=tools_list, request=request)
# print(result_columns)


In [3]:
from langchain.tools import GooglePlacesTool
places = GooglePlacesTool()
places.run("al fornos")

ValidationError: 1 validation error for GooglePlacesAPIWrapper
__root__
  Did not find gplaces_api_key, please add an environment variable `GPLACES_API_KEY` which contains it, or pass  `gplaces_api_key` as a named parameter. (type=value_error)